In [ ]:
import pandas as pd
import numpy as np

pAirports = 'gs://iskldl01-projectby-local-bucket/raw/airports'
pCarriers = 'gs://iskldl01-projectby-local-bucket/raw/carriers'
pFlights = 'gs://iskldl01-projectby-local-bucket/raw/flights'


__Declare Data Sets__

In [ ]:
dFights = sqlContext.read.parquet(pFlights).toPandas()

sAirports = sqlContext.read.csv('gs://iskldl01-projectby-local-bucket/source/airports.csv').toPandas()
dAirports = sqlContext.read.parquet(pAirports).toPandas()

dCarrier = sqlContext.read.parquet(pCarriers).toPandas()

__Airports: Completeness by sum (for numeric columns)__

In [ ]:
headers = sAirports.iloc[0]
sAirports = sAirports[1:]
sAirports.columns = headers


sLat = sAirports['lat'].notnull().sum()
sLong = sAirports['long'].notnull().sum()

rLat = dAirports['lat'].notnull().sum()
rLong = dAirports['longt'].notnull().sum()

if sLat == rLat & sLong == rLong:
    Status = 'Passed'
    badData = ' '
else:
    Status = 'Failed'
    badData = 'Source sum lat: ' + str(sLat) + ' Raw sum: ' + str(rLat) + '; ' + 'Source sum long: ' + str(sLong) + ' Raw sum: ' + str(rLong)

__Flights: Validity by Time Range for ArrTime and ArrDepTime__

In [ ]:
df = dFights[((dFights['ArrTime'] > '2359') & (dFights['ArrTime'] < '0000')) | ((dFights['DepTime'] > '2359') & (dFights['DepTime'] < '0000'))]

if len(df) == 0:
    Vstatus = 'Passed'
    badData1 = ' '
else:
    Vstatus = 'Failed'
    badData1 = df['FlightNum'].tolist()

__Carrier: Uniqueness by PK__

In [ ]:
dupl = dCarrier.groupby(['code']).count()

if len(dupl[dupl['description'] > 1]) > 0:
    Ustatus = 'Failed'
    badData2 = dupl[dupl['description'] > 1].reset_index().values.tolist()
else:
    Ustatus = 'Passed'
    badData2 = ' '
    
print(badData2)

__Flights: Consistency check for CancellationCode__

In [ ]:
df2 = dFights[(dFights['CancellationCode'].isna()) & (dFights['Cancelled'] != 0)]
df3 = dFights[(dFights['CancellationCode'].isin(['A','B','C'])) & (dFights['Cancelled'] != 1)]

if len(df2) != 0 or len(df3) != 0:
    Cstatus = 'Failed'
    badData3 = df3['FlightNum'].tolist() + df2['FlightNum'].tolist()
else:
    Cstatus = 'Passed'
    badData3 = ' '

__Return results as a table in Notebook__

In [ ]:
tableValue = [[1, 'Airports', 'Completness', 'lat, long', Status, badData],
              [2, 'Flights', 'Validity', 'ArrTime, DepTime', Vstatus, badData1],
              [3, 'Carrier', 'Uniqueness', 'code, description', Ustatus, badData2],
              [4, 'Flights', 'Consistency', 'CancellationCode, Cancelled', Cstatus, badData3]]

table = pd.DataFrame(tableValue, columns = ['#', 'Table', 'DQCheck', 'Column', 'Status', 'Bad Data'])

print(table)


__Write results as HTML__

In [ ]:
html = table.to_html()

text_file = open("index.html", "w")
text_file.write(html)
text_file.close()

print("Finish tests")